In [3]:
import time
import torch
from transformers import GPTJForCausalLM, GPT2Tokenizer
import torch

In [4]:
#Will need at least 13-14GB of Vram for CUDA
if torch.cuda.is_available():
    model =  GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B",cache_dir="/datadrive/cache_dir/", torch_dtype=torch.float16).cuda()
else:
    print("GPU not available, check driver and cuda installation and reboot if neccessary")
    #model =  GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.float16)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [7]:
from flask import Flask, request, jsonify
import json

app = Flask(__name__)

@app.route('/generate', methods=['GET']) 
def generate():

    data = json.loads(request.json, strict=False)
    print(data)

    environment_description = data["body"]

    input_text = environment_description
    input_ids = tokenizer.encode(str(input_text), return_tensors='pt').cuda()

    output = model.generate(
        input_ids,
        do_sample=True,
        top_p=0.99,
        top_k=10,
        temperature=0.9,
        max_new_tokens=int(data['token'])*10,
        penalty_alpha=0.6
        )

    out = tokenizer.decode(output[0], skip_special_tokens=True)
    
    
    result = {"result":out}
    
    return jsonify(result)

app.run(port='5002')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002 (Press CTRL+C to quit)
